In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\asus\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
pip install pycaret

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\asus\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\asus\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd

# โหลดข้อมูล
file_path = 'export-4B7B6566022D-1d.xlsx'
dataset = pd.read_excel(file_path, engine="openpyxl")

# ลบแถวที่มีค่า NaN
dataset = dataset.dropna()

# เลือกเฉพาะคอลัมน์ที่เป็นตัวเลข
numeric_cols = dataset.select_dtypes(include=['number']).columns

# คำนวณ IQR สำหรับทุกคอลัมน์
Q1 = dataset[numeric_cols].quantile(0.25)
Q3 = dataset[numeric_cols].quantile(0.75)
IQR = Q3 - Q1

# คำนวณขอบเขตของค่า Outlier
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# กำจัดค่า Outlier
mask = ~((dataset[numeric_cols] < lower_bound) | (dataset[numeric_cols] > upper_bound)).any(axis=1)
cleaned_dataset = dataset.loc[mask]

# บันทึกข้อมูลที่ถูกล้างแล้วเป็นไฟล์ใหม่
output_path = 'cleaned_data.xlsx'
cleaned_dataset.to_excel(output_path, index=False)

# รายงานผล
num_removed = len(dataset) - len(cleaned_dataset)
print(f"ข้อมูลหลังทำความสะอาดเหลือ {len(cleaned_dataset)} แถว จากเดิม {len(dataset)} แถว (ลบไป {num_removed} แถว)")


ข้อมูลหลังทำความสะอาดเหลือ 581 แถว จากเดิม 605 แถว (ลบไป 24 แถว)


In [14]:
import pandas as pd
from pycaret.regression import *

# # โหลดข้อมูลจากไฟล์ Excel
# dataset = pd.read_excel('cleaned_data.xlsx')
files = ['cleaned_data.xlsx', 'eng_cleaned_data.xlsx', 'provinceHatyai_cleaned_data.xlsx']
df_list = [pd.read_excel(file) for file in files]

# รวมข้อมูลทั้งหมดเข้าด้วยกัน
dataset = pd.concat(df_list, ignore_index=True)

# เตรียมข้อมูล
dataset['timestamp'] = pd.to_datetime(dataset['timestamp'], errors='coerce')
dataset.dropna(subset=['timestamp'], inplace=True)

dataset['day'] = dataset['timestamp'].dt.day

# สร้าง Lag Features ทุกๆ 24 ชั่วโมง (1 วัน) จนถึง 168 ชั่วโมง (7 วัน)
for lag in range(1, 8):  
    dataset[f'pm_2_5_lag_{lag}'] = dataset['pm_2_5'].shift(lag)

# สร้าง Rolling Mean และ Rolling Rate (3 วัน, 5 วัน, 7 วัน)
for window in [3]:
    dataset[f'pm_2_5_roll_mean_{window}'] = dataset['pm_2_5'].rolling(window=window).mean()
    dataset[f'pm_2_5_roll_rate_{window}'] = dataset[f'pm_2_5_roll_mean_{window}'].pct_change() * 100

dataset.dropna(inplace=True)

# เลือกคอลัมน์ที่ใช้
features = (
    ['humidity', 'temperature', 'day'] + 
    [f'pm_2_5_lag_{lag}' for lag in range(1, 8)] + 
    [f'pm_2_5_roll_rate_{window}' for window in [3]]
)
target = 'pm_2_5'
dataset_filtered = dataset[features + [target]]

# ตั้งค่า PyCaret
setup(data=dataset_filtered, target=target, fold=10, train_size=0.8, session_id=123, remove_outliers=True)

# เปรียบเทียบโมเดล
best_model = compare_models()

# ใช้ฟังก์ชัน evaluate_model เพื่อตรวจสอบโมเดลที่ดีที่สุด
evaluate_model(best_model)


,Description,Value
0,Session id,123
1,Target,pm_2_5
2,Target type,Regression
3,Original data shape,"(566, 12)"
4,Transformed data shape,"(543, 12)"
5,Transformed train set shape,"(429, 12)"
6,Transformed test set shape,"(114, 12)"
7,Numeric features,11
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,1.6710,5.4361,2.2883,0.9275,0.1425,0.1097,0.1280
gbr,Gradient Boosting Regressor,1.7015,5.3505,2.2897,0.9267,0.1378,0.1100,0.0800
et,Extra Trees Regressor,1.9297,7.4099,2.6936,0.9010,0.1531,0.1246,0.1060
rf,Random Forest Regressor,2.1465,8.9626,2.9643,0.8793,0.1694,0.1391,0.1440
lasso,Lasso Regression,2.4976,12.2233,3.4145,0.8370,0.2129,0.1621,0.0460
llar,Lasso Least Angle Regression,2.4977,12.2234,3.4145,0.8370,0.2129,0.1621,0.0410
en,Elastic Net,2.5032,12.2648,3.4190,0.8363,0.2148,0.1634,0.0440
br,Bayesian Ridge,2.5096,12.4590,3.4445,0.8336,0.2178,0.1653,0.0460
ridge,Ridge Regression,2.5069,12.5070,3.4504,0.8330,0.2192,0.1655,0.0410
lar,Least Angle Regression,2.5069,12.5074,3.4504,0.8330,0.2192,0.1655,0.0520


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [ ]:
# สร้างโมเดล Gradient Boosting Regressor
gbr_model = create_model('gbr')

# ปรับค่าพารามิเตอร์ด้วย tuning - ปรับให้สมดุลระหว่าง R2 และ overfitting
tuned_gbr = tune_model(
    gbr_model,
    optimize='R2',  # ใช้ค่า R-squared เป็นเกณฑ์
    custom_grid = {
        'learning_rate': [0.005, 0.01, 0.02],  # เพิ่ม learning rate เล็กน้อย
        'n_estimators': [50, 75, 100],  # เพิ่มจำนวน trees
        'subsample': [0.5, 0.6, 0.7],  # เพิ่มสัดส่วนข้อมูลที่ใช้
        'max_depth': [3, 4, 5],  # เพิ่มความลึกของต้นไม้เล็กน้อย
        'min_samples_split': [15, 20, 25],  # ปรับให้เหมาะสม
        'min_samples_leaf': [10, 15, 20],  # ปรับให้เหมาะสม
        'max_features': [0.6, 0.7, 0.8],  # เพิ่มจำนวนฟีเจอร์
        'alpha': [0.8, 0.9, 0.95],  # ปรับ regularization
        'min_impurity_decrease': [0.01, 0.02, 0.05]  # ลดค่าขั้นต่ำในการแบ่ง node
    },
    fold=15,  # ลดจำนวน folds ลงเพื่อความเร็ว
    n_iter=60,  # ปรับจำนวนรอบให้เหมาะสม
    early_stopping=True
)

# ประเมินผลโมเดล
evaluate_model(tuned_gbr)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.5564,4.7484,2.1791,0.9422,0.1171,0.0934
1,1.7167,4.7488,2.1792,0.9393,0.1450,0.1251
2,1.2668,2.7920,1.6709,0.9676,0.1139,0.0813
3,2.0138,6.5534,2.5600,0.8796,0.1411,0.1116
4,1.7449,5.1624,2.2721,0.9308,0.1293,0.1149
5,1.9374,7.9727,2.8236,0.9092,0.1379,0.1043
6,1.7868,6.4098,2.5318,0.9300,0.1179,0.0923
7,1.9563,6.6478,2.5783,0.8956,0.1434,0.1210
8,1.7479,4.9158,2.2172,0.9358,0.1690,0.1477


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.1480,22.8339,4.7785,0.7640,0.1769,0.1360
1,2.5680,10.9101,3.3030,0.8150,0.2449,0.2164
2,3.4969,19.5823,4.4252,0.8010,0.2235,0.2063
3,2.7448,11.1454,3.3385,0.7617,0.2920,0.2882
4,2.0277,6.8735,2.6217,0.8535,0.1714,0.1471
5,4.1625,24.7235,4.9723,0.7575,0.2549,0.2310
6,3.0706,15.1356,3.8904,0.7910,0.1848,0.1633
7,2.1534,8.2271,2.8683,0.6808,0.1609,0.1418
8,2.6870,11.6983,3.4203,0.7654,0.2278,0.2157


Fitting 20 folds for each of 60 candidates, totalling 1200 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [21]:
# สร้างโมเดลสุดท้าย
final_model = finalize_model(tuned_gbr)
save_model(tuned_gbr, 'final_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['humidity', 'temperature', 'day',
                                              'pm_2_5_lag_1', 'pm_2_5_lag_2',
                                              'pm_2_5_lag_3', 'pm_2_5_lag_4',
                                              'pm_2_5_lag_5', 'pm_2_5_lag_6',
                                              'pm_2_5_lag_7',
                                              'pm_2_5_roll_rate_3'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('remove_outliers',
                  TransformerWrapper(transformer=RemoveOutliers(random_state=123))),
                 ('trained_model', GradientBoostingRegressor(random_state=123))]),
 'final_model.pkl')

In [24]:
# สร้าง future_df สำหรับการทำนาย 7 วันข้างหน้า
last_timestamp = dataset['timestamp'].max()
future_dates = pd.date_range(start=last_timestamp + pd.Timedelta(days=1), periods=7, freq='D')

# ใช้ค่าเฉลี่ยของ 30 วันล่าสุดสำหรับ humidity และ temperature
recent_data = dataset.sort_values('timestamp').tail(30 * 24)  # 30 วันล่าสุด
avg_humidity = recent_data['humidity'].mean()
avg_temperature = recent_data['temperature'].mean()

# ตรวจสอบว่ามีคอลัมน์ pm_2_5 หรือไม่
if 'pm_2_5' not in dataset.columns:
    raise KeyError("คอลัมน์ 'pm_2_5' ไม่มีอยู่ใน dataset")

# คำนวณ rolling_rate_3 ล่วงหน้า
dataset['pm_2_5_roll_rate_3'] = dataset['pm_2_5'].rolling(window=3*24, min_periods=1).mean()
rolling_rate_3 = dataset['pm_2_5_roll_rate_3']

# ทำนายทีละวัน
predictions = []
current_data = dataset.copy()

for i, date in enumerate(future_dates):
    new_day = pd.DataFrame({
        'timestamp': [date],
        'humidity': [avg_humidity],
        'temperature': [avg_temperature],
        'day': [date.day],
        'hour': [0],
    })

    # สร้าง lag features
    for lag in range(1, 8):
        if i >= lag:
            new_day[f'pm_2_5_lag_{lag}'] = [predictions[i - lag]]
        else:
            idx = -(lag - i) * 24
            if abs(idx) > len(current_data):
                new_day[f'pm_2_5_lag_{lag}'] = [current_data['pm_2_5'].iloc[0]]  # ใช้ค่าตัวแรกของชุดข้อมูล
            else:
                new_day[f'pm_2_5_lag_{lag}'] = [current_data['pm_2_5'].iloc[idx]]

    # ใช้ค่า rolling_rate_3
    if i == 0:
        new_day['pm_2_5_roll_rate_3'] = [rolling_rate_3.iloc[-1]]
    else:
        new_day['pm_2_5_roll_rate_3'] = [sum(predictions[-3:]) / len(predictions[-3:]) if len(predictions) >= 3 else predictions[-1]]

    # ทำนายค่า pm_2_5
    prediction = predict_model(tuned_gbr, data=new_day)
    predicted_value = prediction.iloc[0]['prediction_label']
    predictions.append(predicted_value)

    # เพิ่มข้อมูลที่ทำนายแล้วลงใน current_data
    new_row = new_day.copy()
    new_row['pm_2_5'] = predicted_value
    current_data = pd.concat([current_data, new_row], ignore_index=True)

# สร้าง DataFrame สำหรับผลลัพธ์
result_df = pd.DataFrame({
    'timestamp': future_dates,
    'predicted_pm_2_5': predictions
})

print("ผลลัพธ์การทำนาย:")
print(result_df)


ผลลัพธ์การทำนาย:
            timestamp  predicted_pm_2_5
0 2025-02-12 11:15:42         23.841678
1 2025-02-13 11:15:42         36.269838
2 2025-02-14 11:15:42         45.555336
3 2025-02-15 11:15:42         44.476796
4 2025-02-16 11:15:42         50.454374
5 2025-02-17 11:15:42         51.857535
6 2025-02-18 11:15:42         51.042036


In [25]:
import plotly.graph_objects as go
import pandas as pd

# result_df ใช้ค่าจริงจาก predictions
result_df = pd.DataFrame({
    'timestamp': future_dates,  # ใช้วันที่ที่ทำนาย
    'predicted_pm_2_5': predictions  # ใช้ค่าการทำนายจริงจาก predictions
})

# สร้างกราฟ
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=result_df['timestamp'],
    y=result_df['predicted_pm_2_5'],
    mode='lines+markers',
    name='Predicted PM2.5',
    marker=dict(color='rgba(255, 99, 132, 0.8)', size=10),
    line=dict(width=2, color='rgba(255, 99, 132, 0.8)'),
))

fig.update_layout(
    title='PM2.5 Forecast for the Next 7 Days',
    xaxis_title='Date',
    yaxis_title='PM2.5 (µg/m³)',
    paper_bgcolor='white',
    plot_bgcolor='white',
    xaxis=dict(
        tickmode='array',
        tickvals=result_df['timestamp'],
        ticktext=result_df['timestamp'].dt.strftime('%b %d')
    ),
    yaxis=dict(
        showgrid=True,  # แสดงกริดในแกน y
        zeroline=False,  # ไม่แสดงเส้นที่ 0
    ),
    font=dict(family="Arial, sans-serif", size=12, color='black'),
    hovermode='closest'  # ให้แสดงผลลัพธ์ที่ใกล้ที่สุดเมื่อ hover
)

fig.show()


In [27]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# โหลดค่าจริง (ค่า pm_2_5 ของ 7 วันข้างหน้าจาก dataset จริง)
true_values = dataset.sort_values('timestamp').tail(7)['pm_2_5'].values

# โหลดค่าที่พยากรณ์ได้
predicted_values = result_df['predicted_pm_2_5'].values

# คำนวณค่าความคลาดเคลื่อน
mae = mean_absolute_error(true_values, predicted_values)
mse = mean_squared_error(true_values, predicted_values)
rmse = np.sqrt(mse)
r2 = r2_score(true_values, predicted_values)

# แสดงผลลัพธ์
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R² Score: {r2:.2f}")


Mean Absolute Error (MAE): 17.31
Mean Squared Error (MSE): 357.41
Root Mean Squared Error (RMSE): 18.91
R² Score: -34.25
